OG Credit: https://colab.research.google.com/drive/1AVh9dOsG9DKzfK7gOFrJuitPIcLPqlbO?usp=sharing#scrollTo=g8ntQ9CzRQ-q

In [1]:
!nvidia-smi

Sat Apr  1 19:11:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install -q langchain transformers sentence_transformers llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.0/466.0 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTSimpleVectorIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline


In [4]:
class customLLM(LLM):
   # model_name = "google/flan-t5-large"
    model_name = "ClueAI/ChatYuan-large-v1"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
 
    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())



In [5]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

In [6]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 17.3 MB/s eta 0:00:00


PDF Method: Put/change the online pdf link here:

In [7]:
# !wget http://unec.edu.az/application/uploads/2014/12/pdf-sample.pdf -O sample.pdf
# !wget -U NoSuchBrowser/1.0 http://arxiv.org/pdf/2302.03803.pdf -O sample.pdf
# !wget -U NoSuchBrowser/1.0 https://arxiv.org/pdf/2303.17489.pdf -O sample.pdf
!wget https://aclanthology.org/2022.ccl-1.1.pdf -O sample.pdf

--2023-04-01 19:13:39--  https://aclanthology.org/2022.ccl-1.1.pdf
Resolving aclanthology.org (aclanthology.org)... 174.138.37.75
Connecting to aclanthology.org (aclanthology.org)|174.138.37.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 612105 (598K) [application/pdf]
Saving to: ‘sample.pdf’

sample.pdf          100%[===================>] 597.76K   562KB/s    in 1.1s    

2023-04-01 19:13:41 (562 KB/s) - ‘sample.pdf’ saved [612105/612105]



In [31]:
# https://www.geeksforgeeks.org/extract-text-from-pdf-file-using-python/
# importing required modules
from PyPDF2 import PdfReader
 
# creating a pdf reader object
reader = PdfReader('sample.pdf')
text_pdf = ''
 
# printing number of pages in pdf file
# getting a specific->all page from the pdf file
# extracting text from page   # text = page.extract_text((0,90))
for i in range(len(reader.pages)):
  page = reader.pages[i]
  text_pdf += page.extract_text()
 
# print(text_pdf)

TEXT Method: Put/change the text here:

In [9]:
text1='''未来城市拥有很多不同的功能，如智慧城市、智能交通系统、远程医疗系统、智能家居系统、智慧医疗系统等。未来城市还使用物联网技术，可以远程控制，使用远程操控和传感器来监控城市的发展，帮助他们解决各种城市问题。

未来的城市还采用互联网技术，可以让城市变得更智能，让城市变得更智慧，让每一个人都能更轻松地生活。未来城市的智能设备也可以让城市的使用者更方便地掌握他们的生活，比如智能的智能交通系统和远程医疗系统和智能的安防系统等，可以提供他们更多的便利。
总之，未来城市的愿景是，让所有城市的使用者都能拥有一个美好的未来，让他们可以在自己的城市享受美好的生活。'''

In [10]:
#documents = SimpleDirectoryReader('data').load_data()

from llama_index import Document

text_list = [text1]

documents = [Document(t) for t in text_list]

text_list_pdf = [text_pdf]

documents_pdf = [Document(t) for t in text_list_pdf]

In [11]:
# # set number of output tokens
# num_output = 500
# # set maximum input size
# max_input_size = 512
# # set maximum chunk overlap
# max_chunk_overlap = 15


# prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [12]:
#index = GPTSimpleVectorIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index = GPTListIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index.save_to_disk('index.json')

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
index_pdf = GPTSimpleVectorIndex.from_documents(documents_pdf, service_context=service_context)




In [13]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

TEXT method A&Q:

In [14]:
response = index.query( "未来城市可以做什么？") 


In [15]:
response.response

'答案: 未来城市可以提供很多不同的功能,如智慧城市、智能交通系统、远程医疗系统、智能家居系统、智慧医疗系统等。未来城市还使用物联网技术,可以远程控制,使用远程操控和传感器来监控城市的发展,帮助他们解决各种城市问题。 未来的城市'

In [20]:
response = index.query( "What's the main idea？") 

In [21]:
response.response

'答案: 未来城市的主要理念是: 让城市变得更智能,让城市变得更智慧,让每一个人都能更轻松地生活。 未来城市还采用互联网技术,可以让城市变得更智能,让城市变得更智慧,让每一个人都能更轻松地生活。'

In [84]:
response = index.query( "请描绘一幅画") 

In [85]:
response.response

'。 请画出一幅画 。 请画出一幅画 。 请画出一幅画 。 请画出一幅画 。 请画出一幅画 。 请画出一幅画 。 请画出一幅画 。 请画出一幅画 。 请画出一幅画 。'

PDF method A&Q:

In [42]:
response_pdf = index_pdf.query( "这篇文章的作者是谁？") 

In [43]:
response_pdf.response

'作者: 胡晓丹 '

In [80]:
response_pdf = index_pdf.query( "这篇文章讲了什么？") 

In [81]:
response_pdf.response

'本文 的 研究 对象 是 '

In [78]:
response_pdf = index_pdf.query( "词汇如何变化？") 

In [79]:
response_pdf.response

'------------ 在 " 语言生活 " 中, " 语言生活 " 我们有机会对 " 词汇如何变化 " 进行更深入的探讨。 ------------ 在 "'

In [82]:
response_pdf = index_pdf.query( "请写一首诗") 

In [83]:
response_pdf.response

'------------ 我们有 " 语言 " 的热爱。 We have provided an existing answer: , 以表达对 " 语言 " 的热爱。'

In [93]:
# temp fix for running shell commands on Google Colab

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [87]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 7.4 MB/s eta 0:0

In [88]:
import gradio as gr

In [94]:
index = None

In [95]:
def build_the_bot(input_text):
  text_list = [input_text]
  documents = [Document(t) for t in text_list]
  global index
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
  index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
  return('Index saved successfull!!!')

In [97]:

def chat(chat_history, user_input):
  
  bot_response = index.query(user_input)
  #print(bot_response)
  response = ""
  for letter in ''.join(bot_response.response): #[bot_response[i:i+1] for i in range(0, len(bot_response), 1)]:
      response += letter + ""
      yield chat_history + [(user_input, response)]

In [98]:

with gr.Blocks() as demo:
    gr.Markdown('# Q&A Bot with Hugging Face Models')
    with gr.Tab("Input Text Document"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button = gr.Button("Build the Bot!!!")
        text_button.click(build_the_bot, text_input, text_output)
    with gr.Tab("Knowledge Bot"):
#          inputbox = gr.Textbox("Input your text to build a Q&A Bot here.....")
          chatbot = gr.Chatbot()
          message = gr.Textbox ("What is this document about?")
          message.submit(chat, [chatbot, message], chatbot)

demo.queue().launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://68ae7aad5eb08fa1f6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://68ae7aad5eb08fa1f6.gradio.live
